In [1]:
import pandas as pd 
import numpy as np 


In [2]:
train_df = pd.read_csv("students_500_new.csv")
train_df.head()

,Student_ID,Attendance,Internal_Marks,Semester_Result,Fee_Pending,LMS_Usage,Scholarship,Backlogs,Drop_out
0,1,67,95,55,0,34,0,1,0
1,2,77,95,63,0,21,0,4,1
2,3,83,39,41,0,45,0,1,1
3,4,65,41,55,0,49,0,4,1
4,5,82,70,40,0,20,1,3,1


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Student_ID       500 non-null    int64
 1   Attendance       500 non-null    int64
 2   Internal_Marks   500 non-null    int64
 3   Semester_Result  500 non-null    int64
 4   Fee_Pending      500 non-null    int64
 5   LMS_Usage        500 non-null    int64
 6   Scholarship      500 non-null    int64
 7   Backlogs         500 non-null    int64
 8   Drop_out         500 non-null    int64
dtypes: int64(9)
memory usage: 35.3 KB


In [17]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [7]:
x = train_df.drop(["Student_ID","Drop_out"], axis = 1)
y = train_df["Drop_out"]

In [19]:
train_x, test_x,train_y, test_y = train_test_split(x,y,test_size = 0.2, random_state = 42)
model = CatBoostClassifier()
model.fit(train_x,train_y)


Learning rate set to 0.006966
0:	learn: 0.6805573	total: 2.67ms	remaining: 2.67s
1:	learn: 0.6707706	total: 5.32ms	remaining: 2.66s
2:	learn: 0.6603035	total: 7.64ms	remaining: 2.54s
3:	learn: 0.6524170	total: 9.31ms	remaining: 2.32s
4:	learn: 0.6396825	total: 11.1ms	remaining: 2.22s
5:	learn: 0.6302084	total: 13.2ms	remaining: 2.18s
6:	learn: 0.6224818	total: 14.8ms	remaining: 2.09s
7:	learn: 0.6138215	total: 16.7ms	remaining: 2.07s
8:	learn: 0.6052186	total: 18.5ms	remaining: 2.04s
9:	learn: 0.5943383	total: 20.6ms	remaining: 2.04s
10:	learn: 0.5869760	total: 22.8ms	remaining: 2.05s
11:	learn: 0.5777669	total: 24.3ms	remaining: 2s
12:	learn: 0.5683349	total: 26ms	remaining: 1.98s
13:	learn: 0.5589925	total: 27.6ms	remaining: 1.95s
14:	learn: 0.5520132	total: 29.2ms	remaining: 1.92s
15:	learn: 0.5439832	total: 32.4ms	remaining: 1.99s
16:	learn: 0.5367827	total: 34ms	remaining: 1.97s
17:	learn: 0.5287666	total: 35.8ms	remaining: 1.95s
18:	learn: 0.5212481	total: 38.5ms	remaining: 1.99s

In [21]:
preds = model.predict_proba(test_x)[:,1]
roc = roc_auc_score(test_y, preds)
print("ROC AUC Score: ", roc)

ROC AUC Score:  1.0


In [23]:
imp = model.get_feature_importance()
feat_imp = pd.DataFrame({'Feature': train_x.columns, 'Importance': imp})
feat_imp = feat_imp.sort_values('Importance', ascending=False)
print(feat_imp)

           Feature  Importance
6         Backlogs   34.680263
0       Attendance   24.225394
3      Fee_Pending   20.973875
5      Scholarship   11.211005
1   Internal_Marks    7.685121
4        LMS_Usage    0.690794
2  Semester_Result    0.533549


In [24]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)